# Make a mosiac of Aqu-East and Ser-Main
## Created by Ramsey Karim on July 27, 2022

The goal here is to see if Ser-Main (obsID 1342206676) is salvageable despite the histogram that shows several competitive offset peaks.
Aqu-East has a well defined offset and is adjacent to Ser-Main, so we will try a couple different Ser-Main offsets and make mosaics with the calibrated Aqu-East map and see which is more continuous along the shared edge. Ideally, the "correct" offset will show no major jump across the observation boundary.

This file (currently) resides in the `helpss_scratch_work` directory which is my GitHub repo for HELPSS work, but I will run this file via softlink from the Ser-Main directory: `/n/sgraraid/filaments/Serpens-Aquila/Herschel/processed/1342206676`. **If this notebook is not run from that directory, it will not work correctly!**

In [ ]:
# Get the filenames for the 160 micron observations
import glob
import os.path

ser_main_fn = os.path.abspath(glob.glob("PACS160um-im*-conv.fits").pop())
aqu_east_fn = os.path.abspath(glob.glob("../1342206694/PACS160um-im*-conv.fits").pop())

The game here is that we will make 2 mosaics, one for each potentially viable Ser-Main offset. The two possible Ser-Main offsets are approximatly 100 and 120. From the difference map shown below, we see that 120 corresponds to a broad area surrounding the bright emission, and 100 corresponds to the bright emission region.

In [ ]:
# Gather the offsets to be used (the images selected above are uncalibrated but have offsets identified)
offset_aqu_east = 213.4 # from the summary_table.txt
offset_ser_main = 150.0 # select either 100.0 or 120.0 from the images below

from IPython.display import Image
Image("Difference_Histogram_PACS160um.png")

In [ ]:
Image("Predicted_and_Observed_Flux_PACS160um.png")

In [ ]:
# Load the files and add the offsets
from astropy.io import fits

with fits.open(aqu_east_fn) as hdul:
    aqu_east_hdu = hdul[0].copy()
aqu_east_hdu.data += offset_aqu_east
with fits.open(ser_main_fn) as hdul:
    ser_main_hdu = hdul[0].copy()
ser_main_hdu.data += offset_ser_main

In [ ]:
# Compute the mosaic WCS
from reproject.mosaicking import find_optimal_celestial_wcs

mosaic_wcs, mosaic_shape = find_optimal_celestial_wcs([aqu_east_hdu, ser_main_hdu])
print("Mosaic shape", mosaic_shape)

In [ ]:
# Mosaic and save the result
savename = f"mosaic_AquEast_SerMain_offset_{offset_ser_main:.1f}.fits"
header = mosaic_wcs.to_header()
header['AUTHOR'] = "Ramsey Karim, July 27, 2022"
header['COMMENT'] = "Mosaic of Ser-Main and Aqu-East"
header['COMMENT'] = f"Aqu-East offset {offset_aqu_east:.1f}, Ser-Main offset {offset_ser_main:.1f}"

from reproject import reproject_interp
from reproject.mosaicking import reproject_and_coadd

mosaic_array, footprint = reproject_and_coadd([aqu_east_hdu, ser_main_hdu], mosaic_wcs, shape_out=mosaic_shape,
                                             reproject_function=reproject_interp, match_background=False)
save_hdu = fits.PrimaryHDU(data=mosaic_array, header=header)
save_hdu.writeto(savename)